# 学習済みの分散表現（日本語モデル）の利用 (2)

spaCyの日本語モデルをgensimに取り込み、gensimのAPIで利用

- 単語の類似度
- 類似単語の検索
- 単語の類推 (Word Analogy)
- 仲間はずれ探し
- 短い文の類似度

### ソフトウェアのインストール

gensim
```
conda install gensim python-levenshtein
```
- https://radimrehurek.com/gensim/
- https://anaconda.org/anaconda/gensim

In [1]:
import spacy
from gensim.models.keyedvectors import KeyedVectors

# Spacyの日本語のモデルをロード
nlp = spacy.load('ja_core_news_lg')

# Spacyのモデルをgensimに取り込む
word_list =[]
vector_list = []

# Spacyのモデルの単語とベクトルと取得
for key in nlp.vocab.vectors:
    word_list.append(nlp.vocab.strings[key])
    vector_list.append(nlp.vocab.vectors[key])

# gensimへの取り込み
model = KeyedVectors(nlp.vocab.vectors_length)
model.add_vectors(word_list, vector_list)

### 単語の類似度

In [2]:
model.similarity('猫', '犬')

0.72710896

### 類似単語の検索

In [3]:
model.most_similar('猫')

[('にゃんこ', 0.8090305328369141),
 ('子猫', 0.7787354588508606),
 ('飼い猫', 0.7328758239746094),
 ('犬', 0.7271090149879456),
 ('にゃん', 0.7149522304534912),
 ('野良猫', 0.7057712078094482),
 ('ニャン', 0.7000690698623657),
 ('愛猫', 0.6995570659637451),
 ('三毛', 0.6949921250343323),
 ('家猫', 0.6937754154205322)]

### 単語の類推 (Word Analogy)

In [4]:
# 男にとっての王は、女にとっては何か？
# king - man + woman
model.most_similar(positive=['王', '女'], negative=['男'])

[('王女', 0.5787477493286133),
 ('王様', 0.5546433329582214),
 ('王妃', 0.5514032244682312),
 ('王家', 0.5387033820152283),
 ('女王', 0.532050609588623),
 ('大王', 0.5048933625221252),
 ('国王', 0.5021144151687622),
 ('妃', 0.5001500844955444),
 ('王位', 0.4905719459056854),
 ('聖王', 0.4854174852371216)]

In [5]:
# 男にとっての王は、女にとっては何か？
king = model['王']
man = model['男']
woman = model['女']

# 類推したベクトル
analogy = king - man + woman

# 類推結果に類似する単語
model.similar_by_vector(analogy)

[('王', 0.9089677333831787),
 ('王女', 0.6004545092582703),
 ('王様', 0.5963025689125061),
 ('王妃', 0.5767136812210083),
 ('王家', 0.5543460249900818),
 ('女王', 0.5533928275108337),
 ('国王', 0.5370110273361206),
 ('大王', 0.5363385081291199),
 ('王位', 0.5277222394943237),
 ('妃', 0.5097085237503052)]

### 仲間はずれ探し

In [6]:
model.doesnt_match(('cat', 'dog', 'car'))

'car'

### 短い文の類似度

gensim ではテキストの類似度を計算できないが、単語の集合間の類似度は計算できる。

In [7]:
model.n_similarity(['犬', '可愛い'], ['猫', '可愛い'])

0.9043428

# その他の学習済みモデル

日本語 Wikipedia エンティティベクトル（東北大学）
- http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/

Facebook
- https://fasttext.cc/docs/en/crawl-vectors.html

朝日新聞単語ベクトル
- https://cl.asahi.com/api_data/wordembedding.html

In [8]:
model.most_similar('コロナ')

[('プラズマ', 0.4164362847805023),
 ('パルボウイルス', 0.3846541941165924),
 ('フレア', 0.38262057304382324),
 ('corona', 0.37614893913269043),
 ('パルサー', 0.37256649136543274),
 ('スパーク', 0.37229326367378235),
 ('インフルエンザウィルス', 0.3701728582382202),
 ('ラズマ', 0.3674834370613098),
 ('磁気圏', 0.3592141568660736),
 ('アリオン', 0.35874685645103455)]

In [9]:
model.most_similar(positive=['首相', '米国'], negative=['日本'])

[('大統領', 0.71452796459198),
 ('米大統領', 0.7036280035972595),
 ('外相', 0.6789396405220032),
 ('長官', 0.6777855753898621),
 ('副大統領', 0.6705291271209717),
 ('バイデン', 0.6682686805725098),
 ('オバマ', 0.6670960187911987),
 ('国務', 0.6610805988311768),
 ('クリントン', 0.6603100299835205),
 ('閣僚', 0.6554257273674011)]